In [5]:
# import kagglehub

In [6]:
# path = kagglehub.dataset_download("awsaf49/brats2020-training-data")

# print("Path to dataset files:", path)

In [7]:
# import shutil
# shutil.move(path, "./Data/BRATS")

In [ ]:
from utils.train import warnings, set_seeds
from Network.Unet import Custom3DUnet
import torch
from utils.CustomDataset import BRATS
from torch.utils.data import random_split, DataLoader
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
from skimage import measure
from matplotlib.patches import Polygon
import SimpleITK as sitk
import numpy as np
import random
def seeding(num):
    torch.manual_seed(num)
    torch.cuda.manual_seed(num)
    torch.cuda.manual_seed_all(num)
    np.random.seed(num)
    random.seed(num)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True

seeding(2025)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
skip = False
index=0
depth=37

In [9]:
diceLoss = smp.losses.DiceLoss(
   mode="multiclass",          # For multi-class segmentation
   classes=None,               # Compute the loss for all classes
   log_loss=False,             # Do not use log version of Dice loss
   from_logits=True,           # Model outputs are raw logits
   smooth=1e-5,                # A small smoothing factor for stability
   ignore_index=None,          # Don't ignore any classes
   eps=1e-7                    # Epsilon for numerical stability
)
class Hausdolf95(torch.nn.Module):
    def __init__(self):
        super(Hausdolf95, self).__init__()
        self.distancef = torch.nn.PairwiseDistance()
    def distance(self, x, y):
        return self.distancef(x, y)
        
    def forward(self, x, y):
        maximum_d=torch.max(self.distance(x, y))
        return maximum_d
hausdolf95 = Hausdolf95()
jaccardf = smp.losses.JaccardLoss(
   mode="multiclass",          # For multi-class segmentation
   classes=None,               # Compute the loss for all classes
   log_loss=False,             # Do not use log version of Dice loss
   from_logits=True,           # Model outputs are raw logits
   smooth=1e-5,                # A small smoothing factor for stability          
   eps=1e-7
)

In [10]:
def evaluation(net, loader, diceLoss, HF95, jaccard,DEVICE):
    net.eval()
    net.to(DEVICE)
    dicef= diceLoss.to(DEVICE)
    hf95f = HF95.to(DEVICE)
    jaccardf = jaccard.to(DEVICE)
    dice = 0
    hf95 = 0
    jacc = 0
    length = len(loader)
    for sample in loader:
        X= torch.stack([s["x"] for s in sample], 0)
        Y= torch.stack([s["y"] for s in sample], 0)
        out = net(X.type(torch.float32).unsqueeze(0).to(DEVICE)) 
        dice += (1-dicef(out.type(torch.float32).to(DEVICE), Y.type(torch.int64).to(DEVICE))).item()
        jacc += (1-jaccardf(out.type(torch.float32).to(DEVICE), Y.type(torch.int64).to(DEVICE))).item()
        hf95 += hf95f(out.squeeze().type(torch.float32).to(DEVICE), torch.nn.functional.one_hot(Y.type(torch.int64).squeeze(), 4).permute(3, 0, 1, 2).type(torch.float32).to(DEVICE)).item()
    return {"dice": dice/length, "hf95":hf95/length, 'jaccard': jacc/length}
def convert_to_one_hot(label, classes = 4):
    return torch.nn.functional.one_hot(torch.Tensor(label).type(torch.int64), classes).detach().numpy()
def display(orient, label, num_classes=4):

    fig, ax=plt.subplots(1, 2)

    # ax[0].imshow(label)
    ax[0].imshow(orient, cmap="gray")
    ax[1].imshow(orient, 'gray')
    for indx in range(4):
        if indx == 0:
            continue
        countour = measure.find_contours(convert_to_one_hot(label, num_classes)[...,indx], 0.7)
        for legion in countour:
            
            y=legion.T[0].copy()
            x=legion.T[1].copy()
            legion[:,0]= x
            legion[:,1]= y
            poly=Polygon(legion, closed=False)
            if indx == 1:
                poly.set_color([0,1,1,0.3])
            if indx == 2:
                poly.set_color([1,0,0,0.3])
            if indx == 3:
                poly.set_color([1,0.95,0,0.3])
            ax[1].add_patch(poly)
    ax[0].set_axis_off()
    ax[1].set_axis_off()
    # ax[2].set_axis_off()
    fig.tight_layout()
    fig.show()
def visualization(net, BRATS, index, depth, DEVICE):
    net.to(DEVICE)
    net.eval()
    ori = sitk.GetImageFromArray(BRATS[index]['x'].detach().numpy()[depth, ...], False)
    lab = sitk.GetImageFromArray(BRATS[index]['y'].type(torch.int32).detach().numpy()[depth ,...], False)
    label = sitk.GetArrayFromImage(lab)
    orient = sitk.GetArrayFromImage(ori)
    out = net(BRATS[index]['x'].unsqueeze(0).unsqueeze(0).to(DEVICE))
    display(orient, label)
    display(orient, out.cpu().squeeze().argmax(0).detach().numpy()[depth])

def visualization2(net, fets2023, index, nslice,DEVICE):
    net.to(DEVICE)
    net.eval()
    ori = sitk.GetImageFromArray(fets2023[index]['x'].detach().numpy(), False)
    lab = sitk.GetImageFromArray(fets2023[index]['y'].type(torch.int32).detach().numpy(), False)
    label = sitk.GetArrayFromImage(lab)
    orient = sitk.GetArrayFromImage(ori)
    out = net(fets2023[index]['x'].unsqueeze(0).unsqueeze(0).to(DEVICE))
    display2(orient.squeeze(), label.squeeze())
    display2(orient.squeeze(), out.cpu().squeeze().argmax(0).detach().numpy())

def display2(orient, label, nslice=15, num_classes=4):
    depth = orient.shape[0]
    ranger = range(nslice, nslice*(depth//nslice), nslice)
    fig, ax=plt.subplots(1, len(ranger))
    fig.set_size_inches(16, 16)
    for axind,i in enumerate(range(nslice, nslice*(depth//nslice), nslice)):
        ax[axind].imshow(orient[i,...], cmap="gray")
    
        for indx in range(4):
            if indx == 0:
                continue
            countour = measure.find_contours(convert_to_one_hot(label[i,...], num_classes)[...,indx], 0.7)
            for legion in countour:
                
                y=legion.T[0].copy()
                x=legion.T[1].copy()
                legion[:,0]= x
                legion[:,1]= y
                poly=Polygon(legion, closed=False)
                if indx == 1:
                    poly.set_color([0,1,1,0.6])
                if indx == 2:
                    poly.set_color([1,0,0,0.6])
                if indx == 3:
                    poly.set_color([0.85,0.75,0,0.6])
                ax[axind].add_patch(poly)
        ax[axind].set_axis_off()
    fig.tight_layout()
    fig.show()

In [11]:
dataset = BRATS("Data/BRATS/3/BraTS2020_training_data/content/client/test")
evaloader = DataLoader(dataset, 1, shuffle=False, collate_fn=lambda x:x)
visualset = BRATS("Data/BRATS/3/BraTS2020_training_data/content/client/test")

In [ ]:
net = Custom3DUnet(1, 4, True, f_maps=4, layer_order="gcr", num_groups=4)
net.load_state_dict(torch.load("Models/CentralBRATS/net.pt"))
net.to(DEVICE)
visualization2(net, visualset, index, 15, DEVICE)
if not skip:
    print(evaluation(net, evaloader, diceLoss, hausdolf95, jaccardf, DEVICE))

In [ ]:
net.load_state_dict(torch.load("Models/FedAvgBRATS/net.pt"))
visualization2(net, visualset, index, depth, DEVICE)
if not skip:
    print(evaluation(net, evaloader, diceLoss, hausdolf95, jaccardf, DEVICE))

In [ ]:
net.load_state_dict(torch.load("Models/FedPIDBRATS/net.pt"))
visualization2(net, visualset, index, depth, DEVICE)
if not skip:
    print(evaluation(net, evaloader, diceLoss, hausdolf95, jaccardf, DEVICE))

In [ ]:
net.load_state_dict(torch.load("Models/FedLWRBRATS/net.pt"))
visualization2(net, visualset, index, depth, DEVICE)
if not skip:
    print(evaluation(net, evaloader, diceLoss, hausdolf95, jaccardf, DEVICE))

In [ ]:
net.load_state_dict(torch.load("Models/FedRefBRATS/net.pt"))
visualization2(net, visualset, index, depth, DEVICE)
if not skip:
    print(evaluation(net, evaloader, diceLoss, hausdolf95, jaccardf, DEVICE))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
 
fedavgframe = pd.read_csv('Result/fedavg/FedAvg_brats.csv')
fedpidframe = pd.read_csv('Result/fedpid/FedPID_brats.csv')
fedlwrframe = pd.read_csv('Result/fedlwr/FedLWR_brats.csv')
fedrefframe = pd.read_csv("Result/fedref/FedRef_brats.csv")
plt.plot(fedavgframe['loss'].to_numpy(), color=(0.5,0,1), label= "Fed-Avg", marker= ".")
plt.plot(fedlwrframe['loss'].to_numpy(), color=(0.5,1,0.9), label= "Fed-LWR", marker= ".")
plt.plot(fedpidframe['loss'].to_numpy(), color=(0.5,1,0), label= "Fed-PID", marker= ".")
plt.plot(fedrefframe['loss'].to_numpy(), color=(1,0,0), label= "Fed-Ref", marker= ".")
plt.legend(fontsize=16)
# plt.xlabel("round")
# plt.ylabel("Loss")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
 
fedavgframe = pd.read_csv('Result/fedavg/FedAvg_brats.csv')
fedpidframe = pd.read_csv('Result/fedpid/FedPID_brats.csv')
fedlwrframe = pd.read_csv('Result/fedlwr/FedLWR_brats.csv')
fedrefframe = pd.read_csv("Result/fedref/FedRef_brats.csv")
plt.plot(fedavgframe['mDice'].to_numpy(), color=(0.5,0,1), label= "Fed-Avg", marker= ".")
plt.plot(fedlwrframe['mDice'].to_numpy(), color=(0.5,1,0.9), label= "Fed-LWR", marker= ".")
plt.plot(fedpidframe['mDice'].to_numpy(), color=(0.5,1,0), label= "Fed-PID", marker= ".")
plt.plot(fedrefframe['mDice'].to_numpy(), color=(1,0,0), label= "Fed-Ref", marker= ".")
plt.legend(fontsize=16)
# plt.xlabel("round")
# plt.ylabel("DiceScore")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
 
fedavgframe = pd.read_csv('Result/fedavg/FedAvg_brats.csv')
fedpidframe = pd.read_csv('Result/fedpid/FedPID_brats.csv')
fedlwrframe = pd.read_csv('Result/fedlwr/FedLWR_brats.csv')
fedrefframe = pd.read_csv("Result/fedref/FedRef_brats.csv")
plt.plot(fedavgframe['mHF95'].to_numpy(), color=(0.5,0,1), label= "Fed-Avg", marker= ".")
plt.plot(fedlwrframe['mHF95'].to_numpy(), color=(0.5,1,0.9), label= "Fed-LWR", marker= ".")
plt.plot(fedpidframe['mHF95'].to_numpy(), color=(0.5,1,0), label= "Fed-PID", marker= ".")
plt.plot(fedrefframe['mHF95'].to_numpy(), color=(1,0,0), label= "Fed-Ref", marker= ".")
plt.legend(fontsize=16)
# plt.xlabel("round")
# plt.ylabel("Housdolf Distance")